In [28]:
import time

# MÉTODO A: Usando .apply() (Más lento, como un bucle)
start_time = time.time()
df['Tax_Apply'] = df['Total_Combined_Charge'].apply(lambda x: x * 0.10)
print(f"⏱️ Tiempo con Apply: {time.time() - start_time:.6f} segundos")

# MÉTODO B: Vectorizado (El estándar de oro en MLOps)
start_time = time.time()
df['Tax_Vectorized'] = df['Total_Combined_Charge'] * 0.10
print(f"🚀 Tiempo con Vectorización: {time.time() - start_time:.6f} segundos \n")

print(df[["Tax_Apply","Tax_Vectorized"]].head(6))


⏱️ Tiempo con Apply: 0.003033 segundos
🚀 Tiempo con Vectorización: 0.000966 segundos 

   Tax_Apply  Tax_Vectorized
0      7.556           7.556
1      5.924           5.924
2      6.229           6.229
3      6.680           6.680
4      5.209           5.209
5      6.761           6.761


In [23]:
# --- 1.4.2 Operación Melt ---

# Vamos a concentrar todos los cargos en una sola columna de 'Tipo' y otra de 'Valor'
df_long = df.melt(
    id_vars=['Phone'], 
    value_vars=['Day Charge', 'Eve Charge', 'Night Charge'],
    var_name='Charge_Type', 
    value_name='Amount'
)

print("--- Dataset en formato LARGO (Melt) ---")
##display(df_long.head(10))

charges_table = df_long.pivot_table(
    values = "Amount",
    index = "Charge_Type",
    aggfunc = "mean"
)

display(charges_table.reset_index())
print("---Segundo metodo groupby---")
display(df_long.groupby("Charge_Type").mean("Amount").reset_index())

--- Dataset en formato LARGO (Melt) ---


,Charge_Type,Amount
0,Day Charge,30.562307
1,Eve Charge,17.083540
2,Night Charge,9.039325


---Segundo metodo groupby---


,Charge_Type,Amount
0,Day Charge,30.562307
1,Eve Charge,17.083540
2,Night Charge,9.039325


In [11]:
# --- 1.4 Operaciones Avanzadas: Pivot Table ---

pivot_calls = df.pivot_table(
    values='CustServ Calls', 
    index='State', 
    columns="Int'l Plan", 
    aggfunc='mean'
)

print("--- Promedio de llamadas a soporte por Estado y Plan ---")
display(pivot_calls.head()) # Mostramos los primeros estados

--- Promedio de llamadas a soporte por Estado y Plan ---


Int'l Plan,no,yes
State,,
AK,1.541667,1.750000
AL,1.569444,1.500000
AR,1.957447,2.125000
AZ,1.639344,0.666667
CA,1.600000,0.500000


In [10]:
# --- 1.3 Completando el MERGE ---

# 1. Creamos dos DataFrames artificiales a partir del original
# Tabla A: Información del cliente y plan
df_info = df[['Phone', 'State', 'Area Code', "Int'l Plan", 'VMail Plan']].copy()

# Tabla B: Datos de consumo y Churn
df_usage = df[['Phone', 'Total_Combined_Charge', 'CustServ Calls', 'Churn_Numeric']].copy()

# 2. El MERGE: Unimos ambas tablas usando el Teléfono como llave
df_final = pd.merge(df_info, df_usage, on='Phone')

print("✅ Merge completado con éxito.")
print(f"Columnas finales: {df_final.columns.tolist()}")
display(df_final.head())

✅ Merge completado con éxito.
Columnas finales: ['Phone', 'State', 'Area Code', "Int'l Plan", 'VMail Plan', 'Total_Combined_Charge', 'CustServ Calls', 'Churn_Numeric']


,Phone,State,Area Code,Int'l Plan,VMail Plan,Total_Combined_Charge,CustServ Calls,Churn_Numeric
0,382-4657,KS,415,no,yes,75.56,1,0
1,371-7191,OH,415,no,yes,59.24,1,0
2,358-1921,NJ,415,no,no,62.29,0,0
3,375-9999,OH,408,yes,no,66.80,2,0
4,330-6626,OK,415,yes,no,52.09,3,0


In [9]:
# --- 1.3 Transformaciones y Agregaciones ---

# 1. Crear una métrica de intensidad de uso
# Sumamos todos los minutos para ver cuánto tiempo total pasa el cliente al teléfono
cols_minutos = ['Day Mins', 'Eve Mins', 'Night Mins', 'Intl Mins']
df['Total_Mins'] = df[cols_minutos].sum(axis=1)

# 2. Agrupación (El corazón del análisis)
# Vamos a comparar el comportamiento de los que se fugan (1) vs los que no (0)
# Queremos ver el promedio de: Cargos, Minutos y Llamadas a Soporte
summary_table = df.groupby('Churn_Numeric').agg({
    'Total_Combined_Charge': 'mean',
    'Total_Mins': 'mean',
    'CustServ Calls': 'mean',
    'Account Length': 'count' # Esto nos da el número de clientes en cada grupo
}).rename(columns={'Account Length': 'Customer_Count'})

print("--- Tabla Comparativa: Leales (0) vs Fugados (1) ---")
display(summary_table)

# 3. Exportar el resultado (Entregable del roadmap)
# Esto crea una carpeta de salida si no existe y guarda el resumen
import os
os.makedirs('../outputs', exist_ok=True)
summary_table.to_csv('../outputs/churn_summary.csv')
print("\n✅ Resumen guardado en 'outputs/churn_summary.csv'")

--- Tabla Comparativa: Leales (0) vs Fugados (1) ---


,Total_Combined_Charge,Total_Mins,CustServ Calls,Customer_Count
Churn_Numeric,,,,
0,58.448807,584.511123,1.449825,2850
1,65.355963,635.255901,2.229814,483



✅ Resumen guardado en 'outputs/churn_summary.csv'


In [8]:
import pandas as pd
import sys

print(f"Versión de Python: {sys.version}")
print(f"Versión de Pandas: {pd.__version__}")

# Prueba de creación de un DataFrame básico
data = {'Módulo': ['Setup', 'Python', 'ML'], 'Estado': ['Listo', 'En proceso', 'Pendiente']}
df = pd.DataFrame(data)
print("\nEstructura del Portfolio:")
print(df)


# 📊 Fundamentos de Pandas - Análisis de Churn
#Objetivo:** Carga inicial y exploración de tipos de datos para el proyecto de predicción de fuga.
#**Fase:** 1.1 del Roadmap.

# 1. Cargar el dataset (ajustado al nombre de tu archivo)
path = "../datasets/churn.csv" 
df = pd.read_csv(path)

# 2. Inspección rápida
print("--- Primeras 5 filas ---")
display(df.head())

print("\n--- Información de tipos y nulos ---")
df.info()

print("\n--- Estadísticas descriptivas ---")
display(df.describe())


# --- 1.2 Limpieza de Datos ---

# A. Revisar si hay duplicados
duplicados = df.duplicated().sum()
print(f"Filas duplicadas encontradas: {duplicados}")
# Si hubiera, los borramos: df = df.drop_duplicates()

# B. Revisar valores nulos (Datos faltantes)
print("\nValores nulos por columna:")
print(df.isnull().sum())

# D. Ver el resultado final de la limpieza
print("\nResumen tras limpieza:")
df.info()

# --- 1.2 Limpieza y Ajuste Real (churn.csv) ---

# A. Crear la columna Total_Combined_Charge usando los nombres EXACTOS de tu imagen
# Nota: Quitamos la palabra 'Total' del inicio de cada nombre
cols_cargos = ['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'] # creamos etiquetas para buscar las columnas 

# Verificamos si las columnas existen antes de sumar (para evitar errores), aunque explicitamente no se esta verificando sabemos que las columnas se llaman tal cual en cols_cargo
df['Total_Combined_Charge'] = df[cols_cargos].sum(axis=1)
print("✅ Nueva columna 'Total_Combined_Charge' creada con éxito.")

# B. Transformar 'Churn?' de Object a Int64
if 'Churn?' in df.columns:
    # Si el texto contiene 'True', ponemos 1, de lo contrario 0
    df['Churn_Numeric'] = df['Churn?'].apply(lambda x: 1 if 'True' in str(x) else 0)
    print("✅ Columna 'Churn?' convertida a 'Churn_Numeric' (int64).")

# C. Limpieza de Area Code (Asegurar que sea número)
df['Area Code'] = pd.to_numeric(df['Area Code'], errors='coerce')

# D. Ver los resultados
print("\n--- MUESTRA DE LAS NUEVAS COLUMNAS ---")
print(df[['Day Charge', 'Total_Combined_Charge', 'Churn_Numeric']].head())

print("\n--- INFO FINAL ---")
df.info()

Versión de Python: 3.10.19 | packaged by Anaconda, Inc. | (main, Oct 21 2025, 16:41:31) [MSC v.1929 64 bit (AMD64)]
Versión de Pandas: 2.3.3

Estructura del Portfolio:
   Módulo      Estado
0   Setup       Listo
1  Python  En proceso
2      ML   Pendiente
--- Primeras 5 filas ---


,State,Account Length,Area Code,Phone,Int'l Plan,VMail Plan,VMail Message,Day Mins,Day Calls,Day Charge,...,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn?
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False.
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False.
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False.
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False.
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False.



--- Información de tipos y nulos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   object 
 5   VMail Plan      3333 non-null   object 
 6   VMail Message   3333 non-null   int64  
 7   Day Mins        3333 non-null   float64
 8   Day Calls       3333 non-null   int64  
 9   Day Charge      3333 non-null   float64
 10  Eve Mins        3333 non-null   float64
 11  Eve Calls       3333 non-null   int64  
 12  Eve Charge      3333 non-null   float64
 13  Night Mins      3333 non-null   float64
 14  Night Calls     3333 non-null   int64  
 15  Night Charge    3333 non-null   float64
 16  Intl Mins       3333 non-null   float64


,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,101.064806,437.182418,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,39.822106,42.371290,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,74.000000,408.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,101.000000,415.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,127.000000,510.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,243.000000,510.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


Filas duplicadas encontradas: 0

Valores nulos por columna:
State             0
Account Length    0
Area Code         0
Phone             0
Int'l Plan        0
VMail Plan        0
VMail Message     0
Day Mins          0
Day Calls         0
Day Charge        0
Eve Mins          0
Eve Calls         0
Eve Charge        0
Night Mins        0
Night Calls       0
Night Charge      0
Intl Mins         0
Intl Calls        0
Intl Charge       0
CustServ Calls    0
Churn?            0
dtype: int64

Resumen tras limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   State           3333 non-null   object 
 1   Account Length  3333 non-null   int64  
 2   Area Code       3333 non-null   int64  
 3   Phone           3333 non-null   object 
 4   Int'l Plan      3333 non-null   object 
 5   VMail Plan      3333 non-null   object 
 6   VMail Messa